In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [1]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [4]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [5]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [6]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    # Backup params
    trained_weights = sess.run(weights)
    trained_biases = sess.run(biases)

Epoch: 0001 cost= 180.346195384
Epoch: 0002 cost= 41.507945387
Epoch: 0003 cost= 26.270339645
Epoch: 0004 cost= 18.582367012
Epoch: 0005 cost= 13.631698751
Epoch: 0006 cost= 10.179405432
Epoch: 0007 cost= 7.589066988
Epoch: 0008 cost= 5.648625531
Epoch: 0009 cost= 4.187187531
Epoch: 0010 cost= 3.173177188
Epoch: 0011 cost= 2.358393707
Epoch: 0012 cost= 1.762267673
Epoch: 0013 cost= 1.317866627
Epoch: 0014 cost= 1.105451876
Epoch: 0015 cost= 0.897506789
Optimization Finished!
Accuracy: 0.9465


In [7]:
### Save parameters, inputs, outputs and targets into JSON files
import numpy as np
import json

inputs_total = 328
#TODO: uncomment
images, targets = mnist.train.next_batch(inputs_total)

def SimpleEncode(ndarray):
    return json.dumps(ndarray.tolist())
#def SimpleDecode(jsonDump):
#    return np.array(json.loads(jsonDump))

param_names = ["W1", "b1", "W2", "b2", "Wout", "bout"]
params = [trained_weights['h1'].transpose(), trained_biases['b1'],
          trained_weights['h2'].transpose(), trained_biases['b2'],
          trained_weights['out'].transpose(), trained_biases['out']]

i = 0
for param_name in param_names:
    print(str(i) + ". Saved param \"" + param_name + "\", shape: ", end='')
    json_string = SimpleEncode(params[i].astype(np.float32))
    print(params[i].shape)
    #%timeit SimpleDecode(json_string)
    with open(param_name + '.json', 'w') as outfile:
        outfile.write(json_string)
        outfile.close
    #print(params[i][0])
    i = i + 1
    
# Test images
json_string = SimpleEncode(images.astype(np.float32))
with open('test_images.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
print(str(i) + ". Saved " + str(inputs_total) + " images, shape: ", end='')
print(images.shape)
i = i + 1
# Test labels
#json_string = SimpleEncode(mnist.test.labels[:inputs_total].astype(np.float32))
#with open('test_labels.json', 'w') as outfile:
#    outfile.write(json_string)
#    outfile.close

### Save Tensorflow's forward propagation results into a JSON file
print("Forward-propagating...")
x = tf.placeholder("float", [None, n_input])

# Construct model
pred = multilayer_perceptron(x, trained_weights, trained_biases)
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Produce outputs
    result = sess.run([pred], feed_dict={x: images})
    
# Print results
print("Output[:2]:")
print(result[0][:2])
print("Output[:2] maxed:")
print([list(decision).index(max(decision)) for decision in result[0][:2]])
print("Correct[:2]:")
print([list(decision).index(max(decision)) for decision in targets[0:2]])
    
# Save results
json_string = SimpleEncode(result[0].astype(np.float32))
with open('test_tf_results.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
print(str(i) + ". Saved results, shape: ", end='')
print(result[0].shape)

0. Saved param "W1", shape: (256, 784)
1. Saved param "b1", shape: (256,)
2. Saved param "W2", shape: (256, 256)
3. Saved param "b2", shape: (256,)
4. Saved param "Wout", shape: (10, 256)
5. Saved param "bout", shape: (10,)
6. Saved 328 images, shape: (328, 784)
Forward-propagating...
Output[:2]:
[[ -1.06312305e+03  -4.20236420e+02   2.69117737e+02   5.92790222e+02
    1.67558972e+03   4.27382080e+02  -2.37833069e+02   1.00675531e+03
    5.74043030e+02   1.08903528e+03]
 [ -3.52038757e+02  -9.21647034e+01   2.59646515e+02  -2.41410828e+02
    8.38351562e+02   4.89633026e+02   7.18455887e+01   3.11856934e+02
   -1.28852558e+00   3.68855286e+02]]
Output[:2] maxed:
[4, 4]
Correct[:2]:
[4, 4]
7. Saved results, shape: (328, 10)
